In [ ]:
import cv2
import pytesseract
from pptx import Presentation
from pptx.util import Inches

# Path to Tesseract OCR (ensure it's installed on your system)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to extract key frames from the video
def extract_key_frames(video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frames.append(frame)
        count += 1

    cap.release()
    return frames

# Function to perform OCR on a frame
def extract_text_from_frame(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray_frame)
    return text.strip()

# Function to create PowerPoint presentation
def create_presentation(frames, texts, output_pptx_path):
    prs = Presentation()

    for i, frame in enumerate(frames):
        slide = prs.slides.add_slide(prs.slide_layouts[5])  # Blank slide
        image_path = f"frame_{i}.jpg"
        cv2.imwrite(image_path, frame)

        # Add image to slide
        slide.shapes.add_picture(image_path, Inches(0.5), Inches(0.5), width=Inches(6))

        # Add text to slide
        if texts[i]:
            textbox = slide.shapes.add_textbox(Inches(0.5), Inches(4.5), Inches(6), Inches(1))
            textbox.text = texts[i]

    prs.save(output_pptx_path)

# Main function
def video_to_ppt(video_path, output_pptx_path, frame_interval=30):
    frames = extract_key_frames(video_path, frame_interval)
    texts = [extract_text_from_frame(frame) for frame in frames]
    create_presentation(frames, texts, output_pptx_path)
    print(f"Presentation saved to {output_pptx_path}")

# Example usage
video_path = "example_video.mp4"  # Path to your video file
output_pptx_path = "output_presentation.pptx"  # Output PowerPoint file
video_to_ppt(video_path, output_pptx_path, frame_interval=60)  # Adjust frame_interval as needed
